# 文本预处理

In [1]:
import collections
import re
from d2l import torch as d2l

将数据集读取到由文本行组成的列表中

In [7]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a') # timemachine.txt是一本书

def read_time_machine():  
    """Load the time machine dataset into a list of text lines."""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    # 下面的操作是：把每行 不是大写小写字符，都变成空格，.strip()再把两边空格去掉，.lower()再把大写变小写
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(len(lines))
print(lines[0])
print(lines[10])

3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


每个文本序列被拆分成一个标记列表,也叫做token(tokenization就是通常所说的分词，分出的每一个词语我们把它称为token。)

In [14]:
def tokenize(lines, token='word'):  
    """将文本行拆分为单词或字符标记(token)。"""
    if token == 'word': # 按空格划分成
        return [line.split() for line in lines]
    elif token == 'char': # 按字符划分（就是字母，不完全准确，这里还有空格）
        return [list(line) for line in lines]
    else:
        print('错误：未知令牌类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


构建一个字典，通常也叫做词表（vocabulary），用来将字符串标记映射到从0开始的数字索引中。说白了，就是让每一个token对应一个数字，便于我们送入模型。

In [16]:
class Vocab:  
    """文本词表"""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        """
        min_freq:如果tokens出现的次数少于min_freq，那就直接删掉
        reserved_tokens:指定tokens是句子的开始，还是结束
        """
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        counter = count_corpus(tokens)
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True) # 按token出现的频率，从大到小排（这个是字典的常见排序）
        self.unk, uniq_tokens = 0, ['<unk>'] + reserved_tokens # unk表示token未知，下标记为0
        
        uniq_tokens += [
            token for token, freq in self.token_freqs
            if freq >= min_freq and token not in uniq_tokens]
        
        # self.idx_to_token为给索引返回token，self.token_to_idx为给token返回索引
        self.idx_to_token, self.token_to_idx = [], dict()
        for token in uniq_tokens:
            self.idx_to_token.append(token)
            self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self): # 返回总长度
        return len(self.idx_to_token)

    def __getitem__(self, tokens): # 给token，返回下标
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices): # 给一些下标，返回对应的token
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

def count_corpus(tokens):  
    """统计标记的频率。"""
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens) # 给一堆token，看每一个独一无二的tokens出现的次数

构建词汇表

In [19]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


将每一行文本转换成一个数字索引列表

In [20]:
for i in [0, 10]:
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

words: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices: [1, 19, 50, 40, 2183, 2184, 400]
words: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
indices: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


将所有内容打包到load_corpus_time_machine函数中

In [39]:
def load_corpus_time_machine(max_tokens=-1):  
    """返回时光机器数据集的标记索引列表和词汇表。"""
    lines = read_time_machine() # 将文件读进来，处理一下，并返回成列表
    tokens = tokenize(lines, 'word') # 划分成token
    vocab = Vocab(tokens) # 生成词汇表
    corpus = [vocab[token] for line in tokens for token in line] # 每行的token对应一个下标，然后都存入一个列表中，即corpus
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(32775, 4580)

In [40]:
corpus

[1,
 19,
 50,
 40,
 2183,
 2184,
 400,
 2,
 1,
 19,
 71,
 16,
 37,
 11,
 115,
 42,
 680,
 6,
 586,
 4,
 108,
 7,
 1420,
 5,
 2185,
 587,
 6,
 126,
 25,
 330,
 127,
 439,
 3,
 2186,
 3,
 25,
 1044,
 362,
 113,
 7,
 1421,
 3,
 1045,
 1,
 148,
 588,
 825,
 3,
 1,
 244,
 2187,
 4,
 1,
 2188,
 2189,
 8,
 1,
 2190,
 4,
 589,
 363,
 1,
 2191,
 9,
 826,
 3,
 288,
 8,
 75,
 2192,
 75,
 1422,
 274,
 25,
 2193,
 2194,
 3,
 1423,
 126,
 230,
 131,
 2195,
 6,
 42,
 186,
 35,
 3,
 26,
 7,
 9,
 2196,
 116,
 289,
 1424,
 78,
 76,
 2197,
 2198,
 827,
 4,
 1,
 2199,
 4,
 2200,
 3,
 27,
 132,
 11,
 6,
 126,
 8,
 21,
 114,
 1425,
 1,
 1046,
 17,
 5,
 1426,
 1427,
 14,
 55,
 186,
 3,
 2201,
 2202,
 25,
 2203,
 79,
 21,
 153,
 681,
 14,
 55,
 76,
 11,
 3,
 25,
 2204,
 22,
 85,
 440,
 13,
 590,
 2,
 502,
 28,
 6,
 2205,
 31,
 53,
 154,
 1047,
 9,
 91,
 180,
 2206,
 682,
 1,
 828,
 16,
 364,
 29,
 1428,
 22,
 15,
 2207,
 38,
 2208,
 23,
 5,
 2209,
 38,
 33,
 9,
 230,
 5,
 275,
 69,
 6,
 829,
 126,
 6,
 830,
 